In [20]:
import pandas as pd

In [21]:
# Beispielhafte Daten einlesen (ersetzen Sie 'Airbnb_Open_Data.csv' durch den Pfad zu Ihrer CSV-Datei)
data = pd.read_csv('Airbnb_Open_Data.csv', low_memory=False)

# Überprüfen Sie die ersten Zeilen des DataFrames, um sicherzustellen, dass die Spalten korrekt eingelesen wurden
#print(data.head())

# Funktion zum Bereinigen von Preis- und Gebühren-Spalten (entfernt '$' und konvertiert in float)
def clean_currency(column):
    return column.replace({'\$': '', ',': ''}, regex=True).astype(float)

# Bereinigung und Konvertierung der relevanten Spalten in numerische Typen
data['price'] = clean_currency(data['price'])
data['service fee'] = clean_currency(data['service fee'])
data['reviews per month'] = pd.to_numeric(data['reviews per month'], errors='coerce')
data['review rate number'] = pd.to_numeric(data['review rate number'], errors='coerce')
data['calculated host listings count'] = pd.to_numeric(data['calculated host listings count'], errors='coerce')
data['availability 365'] = pd.to_numeric(data['availability 365'], errors='coerce')

# Ersetzen Sie fehlende Werte durch 0 oder einen geeigneten Wert
numeric_columns = ['price', 'service fee', 'reviews per month', 'review rate number', 'calculated host listings count', 'availability 365']
for column in numeric_columns:
    data[column] = data[column].fillna(0)

# Überprüfen Sie die Datentypen, um sicherzustellen, dass die Konvertierung erfolgreich war
#print(data.dtypes)

# Berechnung des Scores für jeden Eintrag im Datensatz
def calculate_score(row):
    # Gewichtung Faktoren
    alpha = 0.4
    beta = 0.1
    gamma = 0.2
    delta = 0.2
    epsilon = 0.1
    
    price = row['price']
    service_fee = row['service fee']
    reviews_per_month = row['reviews per month']
    review_rate_number = row['review rate number']
    calculated_host_listings_count = row['calculated host listings count']
    availability_365 = row['availability 365']
    
    # Berechnung des Scores
    score = (alpha * price + 
             beta * service_fee + 
             gamma * (reviews_per_month * review_rate_number) + 
             delta * calculated_host_listings_count + 
             epsilon * availability_365)
    return score

# Anwendung der Scoring-Funktion auf den Datensatz
data['score'] = data.apply(calculate_score, axis=1)

# Aggregation der Scores pro Host
host_scores = data.groupby('host id').agg({
    'score': 'sum',
    'host name': 'first',
    'calculated host listings count': 'sum',
    'price': 'mean'
}).reset_index()

# Sortierung der Hosts nach dem aggregierten Score
host_scores = host_scores.sort_values(by='score', ascending=False)

# Top 10 Hosts ausgeben
top_hosts = host_scores.head(10)
print(top_hosts)


# Zähle die Häufigkeit jeder host id
host_id_counts = data['host id'].value_counts()

# Filtere die host id's, die mehr als einmal vorkommen
duplicate_host_ids = host_id_counts[host_id_counts > 1]

# Anzahl der doppelten host id's
num_duplicate_host_ids = len(duplicate_host_ids)
print(len(data.groupby('host id').filter(lambda x : x['host id'].shape[0]>=2)))
print(data)
#print(f"Anzahl der doppelten host ids: {num_duplicate_host_ids}")


           host id     score host name  calculated host listings count   price
78198  75670762826  1119.688       Kaz                           322.0  1169.0
24933  24018604959  1082.680       Kaz                           322.0  1132.0
28438  27445218777  1073.280      Rawn                             6.0  1195.0
23135  22278210133  1054.200     Byron                             6.0  1185.0
12271  11689016852  1053.660   Andrzej                             2.0  1177.0
74669  72235878844  1048.400    Sonder                           192.0  1154.0
33814  32502622332  1041.792      Brik                             2.0  1172.0
65569  63405539422  1040.616     Karen                            18.0  1162.0
65432  63234218406  1038.228      Luis                             8.0  1144.0
54071  52111041287  1035.000    Sonder                           192.0  1114.0
1084
             id                                              NAME  \
0       1001254                Clean & quiet apt home by 

In [33]:
import pandas as pd
import numpy as np
import plotly.express as px

names = ["Alex", "Ramin", "Lennart", "Theo", "Ralf", "Luca", "Annika", "Marius", "Kuz", "Charlie", "Alice", "Eva", "Will", "Hannah", "Mara", "Aaron"]
styles = ['open-street-map','white-bg','carto-positron','carto-darkmatter','stamen-terrain','stamen-toner']

# dummy
np.random.seed(0)
data = {
    'latitude': np.random.uniform(40.5, 40.9, 10),
    'longitude': np.random.uniform(-74.25, -73.75, 10),
    'money': np.random.uniform(100, 1200, 10),
    'name': np.random.choice(names, 10)
}
df = pd.DataFrame(data)
print(df)

px.scatter_mapbox(df, 
                  lat = 'latitude', 
                  lon = 'longitude', 
                  mapbox_style=styles[0],
                  width = 1200,
                  height=600,
                  zoom=9,
                  color='name',
                  size = 'money', size_max=30)

    latitude  longitude        money     name
0  40.719525 -73.854137  1176.480176     Theo
1  40.786076 -73.985553   979.074421     Mara
2  40.741105 -73.965978   607.627298      Eva
3  40.717953 -73.787202   958.582094  Lennart
4  40.669462 -74.214482   230.101868   Marius
5  40.758358 -74.206435   803.913123     Will
6  40.675035 -74.239891   257.688616  Lennart
7  40.856709 -73.833690  1139.135809     Alex
8  40.885465 -73.860922   674.033154     Alex
9  40.653377 -73.814994   556.128134     Ralf
